# Segmenting and Clustering of neighborhoods of Westminster in London city

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
!conda install -c conda-forge geopy --yes

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
	
# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 

import folium # map rendering library

print('Libraries imported.')



Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [2]:
# webscraping to create organized data frame. 
from bs4 import BeautifulSoup
url = "https://en.wikipedia.org/wiki/List_of_London_boroughs"
res = requests.get(url).text
soup = BeautifulSoup(res,'lxml')

df_list = []

for items in soup.find('table', class_= 'wikitable sortable').find_all('tr')[1::]:
    data = items.find_all(['td'])
    data1 = data[0]
    data8 = data[8]    
    try:
        borough_name = data1.get_text()
        borough_name = borough_name[0]
        ll = data8.get_text()
        lat_long = ll[2]
        latitude = lat_long[0]
        longitude = lat_long[1]
#                                                                
        df_list.append((borough_name, latitude, longitude))
    except IndexError:pass

### Extracting latitude and logitude of all Boroughs in London

In [3]:
# web scraping to create organized data frame. 
url = "https://en.wikipedia.org/wiki/List_of_London_boroughs"
res = requests.get(url).text
soup = BeautifulSoup(res,'lxml')

df_list = []

for items in soup.find('table', class_= 'wikitable sortable').find_all('tr')[1::]:
    data = items.find_all(['td'])
    data1 = data[0]
    data8 = data[8]    
    try:
        borough_name = data1.get_text()
        borough_name = borough_name.split('[')
        borough_name = borough_name[0]
        borough_name = borough_name.strip()
        
        ll = data8.get_text()
        ll = ll.split('/')
        lat_long = ll[2]
        lat_long = lat_long.split('(')
        lat_long = lat_long[0]
        lat_long = lat_long.split(';')
        latitude = lat_long[0]
        latitude = latitude.strip()
        longitude = lat_long[1]
        longitude = longitude.strip()
        longitude = longitude.replace(u'\ufeff', '')
        latitude = float(latitude)
        longitude = float(longitude)

        
        #Append the borough name, latitude and logitude
        df_list.append((borough_name, latitude, longitude)) #
    except IndexError:pass

In [4]:
df_borough = pd.DataFrame(df_list, columns=['Borough', 'Latitude' , 'Longitude'])

In [5]:
df_borough.head()

,Borough,Latitude,Longitude
0,Barking and Dagenham,51.5607,0.1557
1,Barnet,51.6252,-0.1517
2,Bexley,51.4549,0.1505
3,Brent,51.5588,-0.2817
4,Bromley,51.4039,0.0198


In [6]:
df_borough.dropna()

,Borough,Latitude,Longitude
0,Barking and Dagenham,51.5607,0.1557
1,Barnet,51.6252,-0.1517
2,Bexley,51.4549,0.1505
3,Brent,51.5588,-0.2817
4,Bromley,51.4039,0.0198
5,Camden,51.5290,-0.1255
6,Croydon,51.3714,-0.0977
7,Ealing,51.5130,-0.3089
8,Enfield,51.6538,-0.0799
9,Greenwich,51.4892,0.0648


### Extracting latitude and longitude of London city. 

In [7]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
address = 'London, UK'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of London City are 51.4893335, -0.144055084527687.


### Map of London with Boroughs.

In [8]:
import folium 

# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough in zip(df_borough['Latitude'], df_borough['Longitude'], df_borough['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
fill_opacity=0.7).add_to(map_london)  
map_london

## Web scraping to find the coordinates of neighborhoods in westminster. 

In [9]:
from urllib.request import urlopen
import re
url = "https://en.wikipedia.org/wiki/List_of_areas_of_London"
res = requests.get(url).text
soup = BeautifulSoup(res,'lxml')

codes = []
areas_list = []
links_list = []
for items in soup.find('table', class_= 'wikitable sortable').find_all('tr')[1::]:
    data = items.find_all(['td'])
    data0 = data[0]
    area_name = data0.text

    data1 = data[1]
    data1 = data1.text
    borough = data1.split('[')
    borough_name = borough[0]
    data5 = data[5]
    code = data5.text
    code = code.strip()
    
    if borough_name == 'Westminster':
        codes.append(code)
        areas_list.append((borough_name,area_name,code))

                
for link in soup.findAll('a', attrs={'href': re.compile("^https://tools.wmflabs.org")}): #using this tool will help us to find the geo coordinates of neighbordhoods of Westminster. 
            htext = link.text
            if htext in codes:
                hlink = link.get('href')
                links_list.append((htext, hlink))

In [10]:

df_westminsterarea = pd.DataFrame(areas_list, columns=['Borough', 'Area', 'Code'])

In [11]:

df_westminsterarea.head()

,Borough,Area,Code
0,Westminster,Aldwych,TQ307810
1,Westminster,Bayswater,TQ255805
2,Westminster,Belgravia,TQ275795
3,Westminster,Charing Cross,TQ305805
4,Westminster,Chinatown,TQ395895


In [12]:
df_westminsterlinks = pd.DataFrame(links_list, columns=['Code','link'])
df_westminsterlinks.head()

,Code,link
0,TQ307810,https://tools.wmflabs.org/geohack/en/51.512815...
1,TQ255805,https://tools.wmflabs.org/geohack/en/51.509500...
2,TQ275795,https://tools.wmflabs.org/geohack/en/51.500066...
3,TQ275795,https://tools.wmflabs.org/geohack/en/51.500066...
4,TQ305805,https://tools.wmflabs.org/geohack/en/51.508368...


In [14]:
columns = df_westminsterlinks.columns.difference(df_westminsterarea.columns)# merging df_westminsterlinks columns to df_westminsterarea columns. 

dfwestminsterarea_links = pd.concat([df_westminsterarea, df_westminsterlinks[columns]], axis=1)
dfwestminsterarea_links.head()

,Borough,Area,Code,link
0,Westminster,Aldwych,TQ307810,https://tools.wmflabs.org/geohack/en/51.512815...
1,Westminster,Bayswater,TQ255805,https://tools.wmflabs.org/geohack/en/51.509500...
2,Westminster,Belgravia,TQ275795,https://tools.wmflabs.org/geohack/en/51.500066...
3,Westminster,Charing Cross,TQ305805,https://tools.wmflabs.org/geohack/en/51.500066...
4,Westminster,Chinatown,TQ395895,https://tools.wmflabs.org/geohack/en/51.508368...


In [15]:
dfwestminsterarea_links.dropna(inplace=True)
dfwestminsterarea_links

,Borough,Area,Code,link
0,Westminster,Aldwych,TQ307810,https://tools.wmflabs.org/geohack/en/51.512815...
1,Westminster,Bayswater,TQ255805,https://tools.wmflabs.org/geohack/en/51.509500...
2,Westminster,Belgravia,TQ275795,https://tools.wmflabs.org/geohack/en/51.500066...
3,Westminster,Charing Cross,TQ305805,https://tools.wmflabs.org/geohack/en/51.500066...
4,Westminster,Chinatown,TQ395895,https://tools.wmflabs.org/geohack/en/51.508368...
5,Westminster,Covent Garden,TQ303809,https://tools.wmflabs.org/geohack/en/51.587093...
6,Westminster,Knightsbridge,TQ275795,https://tools.wmflabs.org/geohack/en/51.512009...
7,Westminster,Lisson Grove,TQ275825,https://tools.wmflabs.org/geohack/en/51.500066...
8,Westminster,Little Venice,TQ262818,https://tools.wmflabs.org/geohack/en/51.527026...
9,Westminster,Maida Vale,TQ255825,https://tools.wmflabs.org/geohack/en/51.521027...


In [16]:
#converting the link in to geo coordinates of longitude and latitude
geo_codes = []
for row in dfwestminsterarea_links.itertuples():
    url = row.link
    code = row.Code
    res = requests.get(url).text
    soup1 = BeautifulSoup(res,'lxml')
    
    for lat in soup1.find('span',{'class':'latitude'}):
        latitude = lat
        latitude = float(latitude)
            
    for long in soup1.find('span',{'class':'longitude'}):    
        longitude = long
        longitude = float(longitude)
        
    geo_codes.append((code, latitude, longitude))

print(geo_codes)

[('TQ307810', 51.512816, -0.117904), ('TQ255805', 51.5095, -0.192977), ('TQ275795', 51.500066, -0.164534), ('TQ305805', 51.500066, -0.164534), ('TQ395895', 51.508369, -0.120969), ('TQ303809', 51.587093, 0.012197), ('TQ275795', 51.51201, -0.123702), ('TQ275825', 51.500066, -0.164534), ('TQ262818', 51.527027, -0.16345), ('TQ255825', 51.521028, -0.182431), ('TQ285815', 51.527474, -0.192265), ('TQ285805', 51.517814, -0.149408), ('TQ295795', 51.508827, -0.149772), ('TQ267814', 51.499612, -0.135737), ('TQ295785', 51.517321, -0.175371), ('TQ295815', 51.490625, -0.136104), ('TQ295805', 51.587093, 0.012197), ('TQ265835', 51.517585, -0.135003), ('TQ295795', 51.508599, -0.13537)]


In [17]:
df_geo_codes = pd.DataFrame(geo_codes, columns=['Code','Latitude','Longitude'])

In [18]:
df_geo_codes

,Code,Latitude,Longitude
0,TQ307810,51.512816,-0.117904
1,TQ255805,51.509500,-0.192977
2,TQ275795,51.500066,-0.164534
3,TQ305805,51.500066,-0.164534
4,TQ395895,51.508369,-0.120969
5,TQ303809,51.587093,0.012197
6,TQ275795,51.512010,-0.123702
7,TQ275825,51.500066,-0.164534
8,TQ262818,51.527027,-0.163450
9,TQ255825,51.521028,-0.182431


In [19]:

columns1 = df_geo_codes.columns.difference(df_westminsterarea.columns) #merging df_geocodes column to df_westminsterarea column

In [20]:
westminster_borough = pd.concat([df_westminsterarea, df_geo_codes[columns1]], axis=1)
westminster_borough.head()

,Borough,Area,Code,Latitude,Longitude
0,Westminster,Aldwych,TQ307810,51.512816,-0.117904
1,Westminster,Bayswater,TQ255805,51.509500,-0.192977
2,Westminster,Belgravia,TQ275795,51.500066,-0.164534
3,Westminster,Charing Cross,TQ305805,51.500066,-0.164534
4,Westminster,Chinatown,TQ395895,51.508369,-0.120969


In [21]:

westminster_borough = westminster_borough.rename(columns={'Area' :'Neighborhood'}) #renaming area to neighborhood

In [22]:
westminster_borough.drop(['Code'], axis=1, inplace=True)

In [23]:
westminster_borough.head()

,Borough,Neighborhood,Latitude,Longitude
0,Westminster,Aldwych,51.512816,-0.117904
1,Westminster,Bayswater,51.509500,-0.192977
2,Westminster,Belgravia,51.500066,-0.164534
3,Westminster,Charing Cross,51.500066,-0.164534
4,Westminster,Chinatown,51.508369,-0.120969


### Extracting the coordinates of Westminster. 

In [24]:
address = 'Westminster, London'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Westminster are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Westminster are 51.4973206, -0.137149.


### Creating a map of Westminster with neighborhoods

In [25]:
map_westminster = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(westminster_borough['Latitude'], westminster_borough['Longitude'], westminster_borough['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_westminster)  
    
map_westminster

#### Define Foursquare Credentials and Version

In [26]:
CLIENT_ID = 'ES2CVBV0FU1WEKZH4OUMNC0OVPY0DCZQX43QD5UVKWBJR0AK' # your Foursquare ID
CLIENT_SECRET = '1JLCYAUKLYPHKTU5BAPYYHYW0CL4RHEQDNBJXPY0MPYARTOP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

### Let's explore the first neighborhood in our dataframe.

#### Get the neighborhood's name.

In [27]:
westminster_borough.loc[0, 'Neighborhood']

'Aldwych'

#### Extracting latitude and longitude of Aldwych

In [28]:
neighborhood_latitude = westminster_borough.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = westminster_borough.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = westminster_borough.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Aldwych are 51.512816, -0.117904.


#### Now, let's get the top 100 venues that are in Aldwych within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [29]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=ES2CVBV0FU1WEKZH4OUMNC0OVPY0DCZQX43QD5UVKWBJR0AK&client_secret=1JLCYAUKLYPHKTU5BAPYYHYW0CL4RHEQDNBJXPY0MPYARTOP&v=20180605&ll=51.512816,-0.117904&radius=500&limit=100'

Send the GET request and examine the resutls

In [30]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d6b0d236f0aa20039047638'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'City of Westminster',
  'headerFullLocation': 'City of Westminster, London',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 200,
  'suggestedBounds': {'ne': {'lat': 51.5173160045, 'lng': -0.1106867142595896},
   'sw': {'lat': 51.5083159955, 'lng': -0.12512128574041037}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bad083df964a52078263be3',
       'name': 'Lyceum Theatre',
       'location': {'address': '21 Wellington St',
        'lat': 51.51159771114791,
        'lng': -0.11978483440420226,
        'labeledLatLngs': [{'label': 'display',
         

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [31]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [32]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

nearby_venues

,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,venue.location.distance,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.neighborhood,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups,venue.venuePage.id
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4bad083df964a52078263be3-0,"[{'id': '4bf58dd8d48988d137941735', 'name': 'T...",4bad083df964a52078263be3,21 Wellington St,GB,London,United Kingdom,NaN,188,"[21 Wellington St, London, Greater London, WC2...","[{'label': 'display', 'lat': 51.51159771114791...",51.511598,-0.119785,NaN,WC2E 7DA,Greater London,Lyceum Theatre,0,[],42261411
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-55c34aac498e536d77bc73b0-1,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",55c34aac498e536d77bc73b0,45 Aldwych,GB,London,United Kingdom,NaN,30,"[45 Aldwych, London, Greater London, WC2B 4DR,...","[{'label': 'display', 'lat': 51.51282288678408...",51.512823,-0.118343,"Covent Garden, Greater London",WC2B 4DR,Greater London,Lundenwic,0,[],NaN
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4ac518e9f964a520d0ab20e3-2,"[{'id': '4bf58dd8d48988d137941735', 'name': 'T...",4ac518e9f964a520d0ab20e3,5 Aldwych,GB,London,United Kingdom,NaN,114,"[5 Aldwych, London, Greater London, WC2B 4LD, ...","[{'label': 'display', 'lat': 51.51227957594698...",51.512280,-0.119322,NaN,WC2B 4LD,Greater London,Novello Theatre,0,[],NaN
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4fc3ad69d5fb6be430de481b-3,"[{'id': '4bf58dd8d48988d119951735', 'name': 'W...",4fc3ad69d5fb6be430de481b,22 Wellington St,GB,London,United Kingdom,NaN,185,"[22 Wellington St, London, Greater London, WC...","[{'label': 'display', 'lat': 51.51194653050263...",51.511947,-0.120193,NaN,WC2,Greater London,Champagne + Fromage,0,[],NaN
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4ede68fdbe7be2833c6a17a2-4,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",4ede68fdbe7be2833c6a17a2,55 Aldwych,GB,London,United Kingdom,NaN,41,"[55 Aldwych, London, Greater London, WC2B 4BB,...","[{'label': 'display', 'lat': 51.5131101279331,...",51.513110,-0.118281,NaN,WC2B 4BB,Greater London,The Delaunay,0,[],121683414
5,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4ac518cff964a52063a620e3-5,"[{'id': '4bf58dd8d48988d171941735', 'name': 'E...",4ac518cff964a52063a620e3,Strand,GB,London,United Kingdom,NaN,226,"[Strand, London, Greater London, WC2R 1LA, Uni...","[{'label': 'display', 'lat': 51.51078575130956...",51.510786,-0.117899,NaN,WC2R 1LA,Greater London,Somerset House,0,[],45884508
6,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-54468f3b498e393cf5d483eb-6,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",54468f3b498e393cf5d483eb,30 Wellington St,GB,London,United Kingdom,NaN,183,"[30 Wellington St, London, Greater London, WC2...","[{'label': 'display', 'lat': 51.5123863702639,...",51.512386,-0.120456,NaN,WC2E 7BD,Greater London,San Carlo Cicchetti,0,[],NaN
7,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4ac518d2f964a5203fa720e3-7,"[{'id': '4bf58dd8d48988d18f941735', 'name': 'A...",4ac518d2f964a5203fa720e3,Somerset House,GB,London,United Kingdom,Strand,139,"[Somerset House (Strand), London, Greater Lond...","[{'label': 'display', 'lat': 51.5115843854343,...",51.511584,-0.117559,NaN,WC2R ORN,Greater London,The Courtauld Gallery,0,[],NaN
8,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b855193f964a520665631e3-8,"[{'id': '4bf58dd8d48988d137941735', 'name': 'T...",4b855193f964a520665631e3,Catherine St,GB,London,United Kingdom,at Russell St,168,"[Catherine St (at Russell St), London, Greater...","[{'label': 'display', 'lat': 51.51306677806735...",51.513067,-0.120304,Covent,WC2B 5JF,Greater London,"Theatre Royal, Drury Lane",0,[],NaN
9,0,

And how many venues were returned by Foursquare?

In [33]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Westminster


#### Let's create a function to repeat the same process to all the neighborhoods in Westminster

In [34]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *westminster_venues*.

In [35]:
westminster_venues = getNearbyVenues(names=westminster_borough['Neighborhood'],
                                   latitudes=westminster_borough['Latitude'],
                                   longitudes=westminster_borough['Longitude']
                                  )

Aldwych
Bayswater
Belgravia
Charing Cross
Chinatown
Covent Garden
Knightsbridge
Lisson Grove
Little Venice
Maida Vale
Marylebone (also St Marylebone)
Mayfair
Millbank
Paddington
Pimlico
Soho
St James's
St John's Wood
Westminster


#### Let's check the size of the resulting dataframe

In [36]:
print(westminster_venues.shape)
westminster_venues.head()

(1372, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aldwych,51.512816,-0.117904,Lyceum Theatre,51.511598,-0.119785,Theater
1,Aldwych,51.512816,-0.117904,Lundenwic,51.512823,-0.118343,Coffee Shop
2,Aldwych,51.512816,-0.117904,Novello Theatre,51.512280,-0.119322,Theater
3,Aldwych,51.512816,-0.117904,Champagne + Fromage,51.511947,-0.120193,Wine Shop
4,Aldwych,51.512816,-0.117904,The Delaunay,51.513110,-0.118281,Restaurant


Let's check how many venues were returned for each neighborhood

In [37]:
westminster_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Aldwych,100,100,100,100,100,100
Bayswater,66,66,66,66,66,66
Belgravia,100,100,100,100,100,100
Charing Cross,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Covent Garden,3,3,3,3,3,3
Knightsbridge,100,100,100,100,100,100
Lisson Grove,100,100,100,100,100,100
Little Venice,41,41,41,41,41,41


#### Let's find out how many unique categories can be curated from all the returned venues

In [38]:
print('There are {} uniques categories.'.format(len(westminster_venues['Venue Category'].unique())))

There are 203 uniques categories.


## 3. Analyze Each Neighborhood

In [39]:
# one hot encoding
westminster_onehot = pd.get_dummies(westminster_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
westminster_onehot['Neighborhood'] = westminster_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [westminster_onehot.columns[-1]] + list(westminster_onehot.columns[:-1])
westminster_onehot = westminster_onehot[fixed_columns]

westminster_onehot.head()

,Yoga Studio,African Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Beer Bar,Beer Garden,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brasserie,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Café,Camera Store,Canal,Candy Store,Cantonese Restaurant,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Currywurst Joint,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Leather Goods Store,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Medical School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,Nightclub,North Indian Restaurant,Opera House,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Paella Restaurant,Pakistani Restaurant,Palace,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pie Shop,Piercing Parlor,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Pub,Radio Station,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Russian Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sri Lankan Restaurant,Stables,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Train Station,Turkish Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Aldwych,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Aldwych,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size.

In [40]:
westminster_onehot.shape

(1372, 203)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [41]:
westminster_grouped = westminster_onehot.groupby('Neighborhood').mean().reset_index()
westminster_grouped

,Neighborhood,Yoga Studio,African Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Beer Bar,Beer Garden,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brasserie,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Café,Camera Store,Canal,Candy Store,Cantonese Restaurant,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Currywurst Joint,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Leather Goods Store,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Medical School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nightclub,North Indian Restaurant,Opera House,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Paella Restaurant,Pakistani Restaurant,Palace,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pie Shop,Piercing Parlor,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Pub,Radio Station,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Russian Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sri Lankan Restaurant,Stables,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Train Station,Turkish Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Aldwych,0.000000,0.00,0.02,0.000000,0.010000,0.01,0.00,0.00,0.000000,0.01,0.000000,0.030000,0.010000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.01,0.01,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.020000,0.03,0.050000,0.00,0.00,0.000000,0.000000,0.01,0.01,0.00,0.00,0.010000,0.010000,0.00,0.04,0.010000,0.000000,0.00,0.01,0.010000,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.00,0.020000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.010000,0.00,0.00,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.02,0.00000,0.040000,0.020000,0.030000,0.020000,0.00,0.00,0.020000,0.00,0.020000,0.00,0.000000,0.000000,0.01,0.00000,0.00,0.010000,0.00000,0.01,0.00,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.000000,0.01,0.00,0.00000,0.000000,0.010000,0.00,0.00,0.00,0.0

#### Let's confirm the new size

In [42]:
westminster_grouped.shape

(19, 203)

#### Let's print each neighborhood along with the top 5 most common venues

In [43]:
num_top_venues = 5

for hood in westminster_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = westminster_grouped[westminster_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aldwych----
          venue  freq
0       Theater  0.10
1  Burger Joint  0.05
2   Coffee Shop  0.05
3    Restaurant  0.04
4  Dessert Shop  0.04


----Bayswater----
                venue  freq
0                 Pub  0.12
1         Coffee Shop  0.08
2       Grocery Store  0.06
3  Italian Restaurant  0.05
4               Hotel  0.05


----Belgravia----
                venue  freq
0            Boutique  0.13
1                Café  0.10
2  Italian Restaurant  0.09
3               Hotel  0.07
4      Clothing Store  0.05


----Charing Cross----
                venue  freq
0            Boutique  0.13
1                Café  0.10
2  Italian Restaurant  0.09
3               Hotel  0.07
4      Clothing Store  0.05


----Chinatown----
          venue  freq
0       Theater  0.09
1         Hotel  0.06
2  Dessert Shop  0.04
3   Coffee Shop  0.03
4      Wine Bar  0.03


----Covent Garden----
                        venue  freq
0                      Lounge  0.33
1          Athletics & Sports  0.33


#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = westminster_grouped['Neighborhood']

for ind in np.arange(westminster_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(westminster_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aldwych,Theater,Coffee Shop,Burger Joint,Hotel,Restaurant,Dessert Shop,Bakery,Ice Cream Shop,Cocktail Bar,Tea Room
1,Bayswater,Pub,Coffee Shop,Grocery Store,Italian Restaurant,Hotel,Chinese Restaurant,Yoga Studio,English Restaurant,Ice Cream Shop,Historic Site
2,Belgravia,Boutique,Café,Italian Restaurant,Hotel,Clothing Store,Japanese Restaurant,Coffee Shop,Jewelry Store,Tea Room,Shoe Store
3,Charing Cross,Boutique,Café,Italian Restaurant,Hotel,Clothing Store,Japanese Restaurant,Coffee Shop,Jewelry Store,Tea Room,Shoe Store
4,Chinatown,Theater,Hotel,Dessert Shop,Wine Bar,Restaurant,Bookstore,Italian Restaurant,Coffee Shop,English Restaurant,Steakhouse


## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [46]:
# set number of clusters
kclusters = 5

westminster_grouped_clustering = westminster_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(westminster_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 2, 0, 1, 0, 2, 3, 4], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [47]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

westminster_merged = westminster_borough

westminster_merged = westminster_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

westminster_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Westminster,Aldwych,51.512816,-0.117904,0,Theater,Coffee Shop,Burger Joint,Hotel,Restaurant,Dessert Shop,Bakery,Ice Cream Shop,Cocktail Bar,Tea Room
1,Westminster,Bayswater,51.509500,-0.192977,0,Pub,Coffee Shop,Grocery Store,Italian Restaurant,Hotel,Chinese Restaurant,Yoga Studio,English Restaurant,Ice Cream Shop,Historic Site
2,Westminster,Belgravia,51.500066,-0.164534,2,Boutique,Café,Italian Restaurant,Hotel,Clothing Store,Japanese Restaurant,Coffee Shop,Jewelry Store,Tea Room,Shoe Store
3,Westminster,Charing Cross,51.500066,-0.164534,2,Boutique,Café,Italian Restaurant,Hotel,Clothing Store,Japanese Restaurant,Coffee Shop,Jewelry Store,Tea Room,Shoe Store
4,Westminster,Chinatown,51.508369,-0.120969,0,Theater,Hotel,Dessert Shop,Wine Bar,Restaurant,Bookstore,Italian Restaurant,Coffee Shop,English Restaurant,Steakhouse


Finally, let's visualize the resulting clusters

In [48]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(westminster_merged['Latitude'], westminster_merged['Longitude'], westminster_merged['Neighborhood'], westminster_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

#### Cluster 1

In [49]:
westminster_merged.loc[westminster_merged['Cluster Labels'] == 0, westminster_merged.columns[[1] + list(range(5,westminster_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aldwych,Theater,Coffee Shop,Burger Joint,Hotel,Restaurant,Dessert Shop,Bakery,Ice Cream Shop,Cocktail Bar,Tea Room
1,Bayswater,Pub,Coffee Shop,Grocery Store,Italian Restaurant,Hotel,Chinese Restaurant,Yoga Studio,English Restaurant,Ice Cream Shop,Historic Site
4,Chinatown,Theater,Hotel,Dessert Shop,Wine Bar,Restaurant,Bookstore,Italian Restaurant,Coffee Shop,English Restaurant,Steakhouse
6,Knightsbridge,Theater,Coffee Shop,Ice Cream Shop,Clothing Store,Dessert Shop,Bakery,Burger Joint,French Restaurant,Sushi Restaurant,Seafood Restaurant
10,Marylebone (also St Marylebone),Bus Stop,Pizza Place,Deli / Bodega,Coffee Shop,Dance Studio,Italian Restaurant,Indian Restaurant,Gym,Greek Restaurant,General Entertainment
11,Mayfair,Sandwich Place,French Restaurant,Italian Restaurant,Juice Bar,Burger Joint,Pub,Coffee Shop,Cosmetics Shop,Bakery,Hotel
17,St John's Wood,Coffee Shop,Cocktail Bar,Tapas Restaurant,Clothing Store,Pizza Place,Hotel,Bakery,Restaurant,Japanese Restaurant,Tea Room
18,Westminster,Clothing Store,Theater,Hotel,Seafood Restaurant,Italian Restaurant,Men's Store,Boutique,Cocktail Bar,Indian Restaurant,Ice Cream Shop


#### Cluster 2 

In [50]:
westminster_merged.loc[westminster_merged['Cluster Labels'] == 1, westminster_merged.columns[[1] + list(range(5, westminster_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Covent Garden,Lounge,Athletics & Sports,Construction & Landscaping,Women's Store,Flea Market,Garden Center,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant
16,St James's,Lounge,Athletics & Sports,Construction & Landscaping,Women's Store,Flea Market,Garden Center,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant


#### Cluster 3 

In [51]:
westminster_merged.loc[westminster_merged['Cluster Labels'] == 2, westminster_merged.columns[[1] + list(range(5, westminster_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Belgravia,Boutique,Café,Italian Restaurant,Hotel,Clothing Store,Japanese Restaurant,Coffee Shop,Jewelry Store,Tea Room,Shoe Store
3,Charing Cross,Boutique,Café,Italian Restaurant,Hotel,Clothing Store,Japanese Restaurant,Coffee Shop,Jewelry Store,Tea Room,Shoe Store
7,Lisson Grove,Boutique,Café,Italian Restaurant,Hotel,Clothing Store,Japanese Restaurant,Coffee Shop,Jewelry Store,Tea Room,Shoe Store


#### Cluster 4 

In [52]:
westminster_merged.loc[westminster_merged['Cluster Labels'] == 3, westminster_merged.columns[[1] + list(range(5, westminster_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Little Venice,Café,Hotel,Theater,Hookah Bar,Pub,Coffee Shop,Grocery Store,Thai Restaurant,Flower Shop,Garden
12,Millbank,Hotel,French Restaurant,Restaurant,Hotel Bar,Café,Seafood Restaurant,Japanese Restaurant,Cocktail Bar,Indian Restaurant,Art Gallery
13,Paddington,Hotel,Coffee Shop,Sandwich Place,Indian Restaurant,Juice Bar,Hotel Bar,Sporting Goods Shop,Café,Modern European Restaurant,Breakfast Spot
14,Pimlico,Hotel,Coffee Shop,Café,Italian Restaurant,Pub,Sandwich Place,Grocery Store,Bar,Outdoor Sculpture,Plaza
15,Soho,Hotel,Café,Pub,Park,Italian Restaurant,Deli / Bodega,Pizza Place,Sandwich Place,Chinese Restaurant,Camera Store


#### Cluster 5 

In [53]:
westminster_merged.loc[westminster_merged['Cluster Labels'] == 4, westminster_merged.columns[[1] + list(range(5, westminster_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Maida Vale,Café,Pub,Seafood Restaurant,Garden,Restaurant,Bar,Canal,Persian Restaurant,Garden Center,Roof Deck
